# BEE 4750 Final Project

Team Members: Bailey Belinger (bcb87), Camila Monter (cm755), Priya Shah (ps742)

In [4]:
# Importing packages
import Pkg
#Pkg.activate(@__DIR__)
#Pkg.instantiate()
#Pkg.add("JuMP")
#Pkg.add("HiGHS")
#Pkg.add("DataFrames")
#Pkg.add("Plots")
#Pkg.add("Measures")
#Pkg.add("CSV")
#Pkg.add("MarkdownTables")
#Pkg.add("LinearAlgebra")

using JuMP
using HiGHS
using DataFrames
using Plots
using Measures
using CSV
using MarkdownTables
using LinearAlgebra

   Resolving package versions...
    Updating `~/bee4750/l3bc-final-project/Project.toml`
  [1862ce21] + MarkdownTables v1.1.0
    Updating `~/bee4750/l3bc-final-project/Manifest.toml`
  [dce04be8] + ArgCheck v2.3.0
  [0b91fe84] + DisplayAs v0.1.6
  [1862ce21] + MarkdownTables v1.1.0
   Resolving package versions...
    Updating `~/bee4750/l3bc-final-project/Project.toml`
  [37e2e46d] + LinearAlgebra
  No Changes to `~/bee4750/l3bc-final-project/Manifest.toml`
[ Info: Precompiling HiGHS [87dc4568-4c63-4d18-b0c0-bb2238e4078b]
┌ Warning: Module MathOptInterface with build ID fafbfcfd-e67b-f785-000d-bef0d4fcc765 is missing from the cache.
│ This may mean MathOptInterface [b8f27783-ece8-5eb3-8dc8-9495eed66fee] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing HiGHS [87dc4568-4c63-4d18-b0c0-bb2238e4078b].
[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
┌ War

In [11]:
##Carbon Sequestration Rates

m2 = 0.092903 #conversion to m^2
#Trees
oak_tree_cs = 0.593 #kg C/m2/year
ash_tree_cs = 5.903 #kgC/m2/year
maple_tree_cs = 0.202 #kg C/m2/year

#Green Roof Systems
grass_cs = 0.2 #kg C/m2
prairie_mix_cs = 4.63 #kg C/m2
herb_garden_cs = 11.03 #kg C/m2

##Costs
#Trees
oak_tree_cost = 42.51 #per tree WHEN YOU BUY 10
ash_tree_cost = 109.75 #per tree
maple_tree_cost = 19.99 #per tree

#Green Roof Systems
installation_cost = 30*m2 #$/sqm
maintenance_cost = 1.50*m2 #$/sqm/year
grass_cost = 0.86*m2 #$/sqm
prairie_mix_cost = 0.03*m2 #$/sqm
herb_garden_cost = 9.50*m2 #$/sqm

#Available areas
city_area = 2526871.39 #m^2
total_area = 4761611.96 #m^2
park_area = 1563736.94 #m^2
usable_roof_area = 0.5 #percent of city that is usable rooftop area

#Emission rates in Chicago
current_emission_level = 0.0517 * total_area #kg CO2 / year
percent_reduction_goal = 30

0.5

In [13]:
air_quality_model = Model(HiGHS.Optimizer)
@variable(air_quality_model, G[1:6]>=0) #area in meters squared

fixed_cost = installation_cost*G[4] +  installation_cost*G[5] +  installation_cost*G[6] + maintenance_cost*G[4] + maintenance_cost*G[5] + maintenance_cost*G[6]
variable_cost = G[1]*oak_tree_cost + G[2]*ash_tree_cost + G[3]*maple_tree_cost + G[4]*grass_cost + G[5]*prairie_mix_cost + G[6]*herb_garden_cost
cost = fixed_cost + variable_cost
@objective(air_quality_model, Min, cost)
print(air_quality_model)

emissions_reduction = (G[1]*oak_tree_cs + G[2]*ash_tree_cs + G[3]*maple_tree_cs + G[4]*grass_cs + G[5]*prairie_mix_cs + G[6]*herb_garden_cs)*(44/12) #kg CO2 / year
new_total_emissions = current_emission_level - emissions_reduction
percent_reduction = (current_emission_level - new_total_emissions)/current_emission_level * 100
@constraint(air_quality_model, emisisons_constraint, percent_reduction >= percent_reduction_goal)
@constraint(air_quality_model, area_constraint_park, G[1]+G[2]+G[3] <= park_area)
@constraint(air_quality_model, area_constraint_rooftop, G[4]+G[5]+G[6] <= usable_roof_area)